In [11]:
!pip install opencv-contrib-python --user


  Using cached opencv_contrib_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.9.0.80-cp37-abi3-win_amd64.whl (45.3 MB)


In [17]:
import os
import cv2
import numpy as np
from skimage.morphology import skeletonize

def apply_pipeline(input_folder, output_folder):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Define functions for each step of the pipeline
    def bilateral_filtering(img):
        return cv2.bilateralFilter(img, 9, 75, 75)

    def qgmf(img):
        qgmf_filter = np.array([[1, 4, 6, 4, 1],
                                [4, 16, 24, 16, 4],
                                [6, 24, 36, 24, 6],
                                [4, 16, 24, 16, 4],
                                [1, 4, 6, 4, 1]]) / 256.0
        return cv2.filter2D(img, -1, qgmf_filter)

    def edge_detection(img):
        return cv2.Canny(img, 100, 200)

    def edge_thinning(img):
        return skeletonize(img)

    def edge_linking(img):
        # Convert binary image to uint8
        img_uint8 = img.astype(np.uint8)

        # Apply Hough line detection
        lines = cv2.HoughLinesP(img_uint8, rho=1, theta=np.pi/180, threshold=50, minLineLength=50, maxLineGap=10)

        # Create a blank image for drawing lines
        linked_img = np.zeros_like(img_uint8)

        # Draw the detected lines
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(linked_img, (x1, y1), (x2, y2), (255, 255, 255), 1)  # Draw white line

        return linked_img

    # Process each image in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            img_path = os.path.join(input_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            # Apply filters
            img_filtered = bilateral_filtering(img)
            img_filtered = qgmf(img_filtered)
            img_edges = edge_detection(img_filtered)
            img_thinned = edge_thinning(img_edges)
            img_linked = edge_linking(img_thinned)

            # Write output images
            cv2.imwrite(os.path.join(output_folder, f'{filename}_filtered.jpg'), img_filtered)
            cv2.imwrite(os.path.join(output_folder, f'{filename}_edges.jpg'), img_edges)
            cv2.imwrite(os.path.join(output_folder, f'{filename}_thinned.jpg'), img_thinned.astype(np.uint8) * 255)  # Convert to uint8 and scale to 0-255
            cv2.imwrite(os.path.join(output_folder, f'{filename}_linked.jpg'), img_linked)

# Example usage:
input_folder =  r"C:\Users\LENOVO\Downloads\archive (5)\CASIA1\Sp"
output_folder = r"C:\Users\LENOVO\Downloads\bicasia"
apply_pipeline(input_folder, output_folder)

